In [55]:
from pybaseball import batting_stats
from pybaseball import playerid_lookup
from pybaseball import player_search_list
from pybaseball import playerid_reverse_lookup
import numpy as np
import pandas as pd
import csv
pd.set_option('display.max_rows', 10)

In [56]:
salary_df = pd.read_csv('mlb-free-agency.csv')
salary_df = salary_df[salary_df['total_salary'].notna()]
salary_df = salary_df.drop([0])

In [57]:
# drop first column and reset index
salary_df.drop(columns=salary_df.columns[0], axis=1, inplace=True)
salary_df.reset_index(drop=True)

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,spotracID,spotracLink
0,Albert Pujols,DH,31.8,STL,LAA,10,240000000.0,24000000.0,2012,795,https://www.spotrac.com/redirect/player/795/
1,Prince Fielder,DH,27.7,MIL,DET,9,214000000.0,23777778.0,2012,493,https://www.spotrac.com/redirect/player/493/
2,Jose Reyes,SS,28.4,NYM,MIA,6,106000000.0,17666667.0,2012,559,https://www.spotrac.com/redirect/player/559/
3,C.J. Wilson,SP,31.0,TEX,LAA,5,77500000.0,15500000.0,2012,874,https://www.spotrac.com/redirect/player/874/
4,Mark Buehrle,SP,32.7,CHW,MIA,4,58000000.0,14500000.0,2012,169,https://www.spotrac.com/redirect/player/169/
...,...,...,...,...,...,...,...,...,...,...,...
1269,Clint Frazier,OF,27.2,NYY,CHC,1,1500000.0,1500000.0,2022,23831,https://www.spotrac.com/redirect/player/23831/
1270,Johan Camargo,3B,27.9,ATL,PHI,1,1400000.0,1400000.0,2022,21154,https://www.spotrac.com/redirect/player/21154/
1271,Jhoulys Chacin,SP,33.8,COL,COL,1,1250000.0,1250000.0,2022,7307,https://www.spotrac.com/redirect/player/7307/
1272,Taylor Clarke,RP,28.5,ARI,KC,1,975000.0,975000.0,2022,17735,https://www.spotrac.com/redirect/player/17735/


In [58]:
# drop all pitchers: SP and RP
drop_SP = salary_df[salary_df['position'] == 'SP'].index
salary_df.drop(drop_SP, inplace = True)
drop_RP = salary_df[salary_df['position'] == 'RP'].index
salary_df.drop(drop_RP, inplace = True)
# replace LF,CF,RF with OF
salary_df.position.replace(['LF', 'CF', 'RF'], 'OF', inplace=True)

In [59]:
salary_df.sort_values('name', inplace=True)
salary_df.reset_index(drop=True, inplace=True)

In [60]:
# check if there's any players who share the same name
# 1. keep entires with unique ID
unique = salary_df.drop_duplicates(subset=['spotracID'])
# 2. check if there's any duplciate name in 'Unique'
unique[unique.duplicated(subset=['name'], keep=False)]
# there isn't any

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,spotracID,spotracLink


## Lookup fangraphs playerID using there name

In [61]:
# create playersearch DataFrame:

player_search = salary_df[['name','spotracID','spotracLink']].copy()
player_search.drop_duplicates(subset=['name'], keep='first', inplace=True)

In [62]:
for i, row in player_search.iterrows():
    player_search.at[i, 'first_name'] = row['name'].split()[0].lower()
    if len(row['name'].split()) <= 2:
        player_search.at[i, 'last_name'] = row['name'].split()[-1].lower()
    else:
        player_search.at[i, 'last_name'] = row['name'].split()[1].lower()

In [63]:
# change a.j. to 'a. j.'

for i, row in player_search.iterrows():
    if '.' in row.first_name:
        row.first_name = row.first_name.replace('.', '. ', 1)
        player_search.at[i, 'first_name'] = row.first_name
player_search

,name,spotracID,spotracLink,first_name,last_name
0,A.J. Ellis,7427,https://www.spotrac.com/redirect/player/7427/,a. j.,ellis
1,A.J. Pierzynski,188,https://www.spotrac.com/redirect/player/188/,a. j.,pierzynski
5,A.J. Pollock,10693,https://www.spotrac.com/redirect/player/10693/,a. j.,pollock
6,Aaron Hill,908,https://www.spotrac.com/redirect/player/908/,aaron,hill
7,Abraham Almonte,14343,https://www.spotrac.com/redirect/player/14343/,abraham,almonte
...,...,...,...,...,...
578,Yonder Alonso,7281,https://www.spotrac.com/redirect/player/7281/,yonder,alonso
579,Yorbit Torrealba,276,https://www.spotrac.com/redirect/player/276/,yorbit,torrealba
580,Yoshitomo Tsutsugo,47079,https://www.spotrac.com/redirect/player/47079/,yoshitomo,tsutsugo
581,Yuniesky Betancourt,749,https://www.spotrac.com/redirect/player/749/,yuniesky,betancourt


In [64]:
# create a list of tuples(last, first) to pass into the player lookup function
name_list = []

for i, row in player_search.iterrows():
    tup = (row.last_name, row.first_name)
    name_list.append(tup)
name_list

[('ellis', 'a. j.'),
 ('pierzynski', 'a. j.'),
 ('pollock', 'a. j.'),
 ('hill', 'aaron'),
 ('almonte', 'abraham'),
 ('duvall', 'adam'),
 ('jones', 'adam'),
 ('kennedy', 'adam'),
 ('laroche', 'adam'),
 ('lind', 'adam'),
 ('rosales', 'adam'),
 ('hechavarria', 'adeiny'),
 ('gonzalez', 'adrian'),
 ('almora', 'albert'),
 ('pujols', 'albert'),
 ('callaspo', 'alberto'),
 ('escobar', 'alcides'),
 ('de', 'alejandro'),
 ('avila', 'alex'),
 ('gonzalez', 'alex'),
 ('gordon', 'alex'),
 ('rios', 'alex'),
 ('ramirez', 'alexei'),
 ('amarista', 'alexi'),
 ('simmons', 'andrelton'),
 ('blanco', 'andres'),
 ('torres', 'andres'),
 ('mccutchen', 'andrew'),
 ('jones', 'andruw'),
 ('pagan', 'angel'),
 ('ramirez', 'aramis'),
 ('cabrera', 'asdrubal'),
 ('jackson', 'austin'),
 ('kearns', 'austin'),
 ('romine', 'austin'),
 ('garcia', 'avisail'),
 ('revere', 'ben'),
 ('zobrist', 'ben'),
 ('hall', 'bill'),
 ('butler', 'billy'),
 ('hamilton', 'billy'),
 ('abreu', 'bobby'),
 ('wilson', 'bobby'),
 ('miller', 'brad'),


In [65]:
playerID = player_search_list(name_list)

In [66]:
playerID

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,ellis,a. j.,454560,ellia001,ellisaj01,5677,2008.0,2018.0
1,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
2,hill,aaron,431094,hilla001,hillaa01,6104,2005.0,2017.0
3,almonte,abraham,501659,almoa001,almonab01,5486,2013.0,2021.0
4,duvall,adam,594807,duvaa001,duvalad01,10950,2014.0,2021.0
...,...,...,...,...,...,...,...,...
381,gomes,yan,543228,gomey001,gomesya01,9627,2012.0,2021.0
382,grandal,yasmani,518735,grany001,grandya01,11368,2012.0,2021.0
383,alonso,yonder,475174,alony001,alonsyo01,2530,2010.0,2019.0
384,betancourt,yuniesky,435358,betay001,betanyu01,8585,2005.0,2013.0


In [67]:
# check which entires contain duplicate names
# remove all players whose mlb_played_last <= 2010

pd.set_option('display.max_rows', 10)
playerID[playerID.duplicated(subset=['name_last', 'name_first'], keep=False)]
idx_drop = playerID[playerID['mlb_played_last']<=2010].index
playerID.drop(idx_drop, inplace=True)

In [68]:
# check if there are still duplicated players

playerID[playerID.duplicated(subset=['name_last', 'name_first'], keep=False)]
# drop chris young 3196 and taylor michael 2591, they are not the ones we're looking for
cy_idx = playerID[playerID['key_fangraphs']==3196].index
mt_idx = playerID[playerID['key_fangraphs']==2591].index
playerID.drop(cy_idx, inplace=True)
playerID.drop(mt_idx, inplace=True)

In [69]:
print(f"Number of players we searched: {len(name_list)}")
print(f"Number of result that we got: {len(playerID.index)}")

Number of players we searched: 383
Number of result that we got: 368


In [70]:
# create a list of player name that returned positive results
name_list_positive = []
for i, row in playerID.iterrows():
    tup = (row.name_last, row.name_first)
    name_list_positive.append(tup)
# check who are we still missing
name_list_missing = [x for x in name_list if x not in name_list_positive]
name_list_missing

[('pollock', 'a. j.'),
 ('de', 'alejandro'),
 ('ramirez', 'alexei'),
 ('lemahieu', 'd. j.'),
 ('santana', 'daniel'),
 ('iglesias', 'jose'),
 ('martin', 'leonys'),
 ('pina', 'manuel'),
 ('upton', 'melvin'),
 ('aoki', 'norichika'),
 ('pearce', 'steven'),
 ('la', 'tommy'),
 ('cespedes', 'yoenis'),
 ('torrealba', 'yorbit'),
 ('tsutsugo', 'yoshitomo')]

In [71]:
# create a 'fullname' column in salary_df for merge with playerID

salary_df['full_name'] = salary_df['name'].str.lower().str.replace(' ', '')

# create a fullname column in playerID

playerID['full_name'] = playerID['name_first'] + playerID['name_last']
playerID['full_name'] = playerID['full_name'].str.lower().str.replace(' ', '')

## Merege salary and playerID

In [72]:
salaryFinalDF = pd.merge(salary_df, playerID, on='full_name', how='inner')
salaryFinalDF

,name,position,age,from_team,to_tam,contract_length,total_salary,avg_salary,year,spotracID,spotracLink,full_name,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,A.J. Ellis,C,35.6,PHI,MIA,1,2500000.0,2500000.0,2017,7427,https://www.spotrac.com/redirect/player/7427/,a.j.ellis,ellis,a. j.,454560,ellia001,ellisaj01,5677,2008.0,2018.0
1,A.J. Pierzynski,C,37.5,BOS,STL,1,500000.0,500000.0,2014,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
2,A.J. Pierzynski,C,35.9,CHW,TEX,1,7500000.0,7500000.0,2013,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
3,A.J. Pierzynski,C,37.9,STL,ATL,1,2000000.0,2000000.0,2015,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
4,A.J. Pierzynski,C,38.8,ATL,ATL,1,3000000.0,3000000.0,2016,188,https://www.spotrac.com/redirect/player/188/,a.j.pierzynski,pierzynski,a. j.,150229,piera001,pierza.01,746,1998.0,2016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,Yasmani Grandal,C,30.2,LAD,MIL,1,18250000.0,18250000.0,2019,8791,https://www.spotrac.com/redirect/player/8791/,yasmanigrandal,grandal,yasmani,518735,grany001,grandya01,11368,2012.0,2021.0
557,Yasmani Grandal,C,31.0,MIL,CHW,4,73000000.0,18250000.0,2020,8791,https://www.spotrac.com/redirect/player/8791/,yasmanigrandal,grandal,yasmani,518735,grany001,grandya01,11368,2012.0,2021.0
558,Yonder Alonso,1B,30.7,SEA,CLE,2,16000000.0,8000000.0,2018,7281,https://www.spotrac.com/redirect/player/7281/,yonderalonso,alonso,yonder,475174,alony001,alonsyo01,2530,2010.0,2019.0
559,Yuniesky Betancourt,1B,31.1,PHI,MIL,1,900000.0,900000.0,2013,749,https://www.spotrac.com/redirect/player/749/,yunieskybetancourt,betancourt,yuniesky,435358,betay001,betanyu01,8585,2005.0,2013.0


In [73]:
# drop unwanted columns
salaryFinalDF.drop(columns=['contract_length', 
                            'total_salary', 
                            'full_name', 
                            'name_last', 
                            'name_first',
                            'key_mlbam', 
                            'key_retro',
                            'key_bbref',
                            'spotracID',
                            'spotracLink'], inplace=True)
salaryFinalDF

,name,position,age,from_team,to_tam,avg_salary,year,key_fangraphs,mlb_played_first,mlb_played_last
0,A.J. Ellis,C,35.6,PHI,MIA,2500000.0,2017,5677,2008.0,2018.0
1,A.J. Pierzynski,C,37.5,BOS,STL,500000.0,2014,746,1998.0,2016.0
2,A.J. Pierzynski,C,35.9,CHW,TEX,7500000.0,2013,746,1998.0,2016.0
3,A.J. Pierzynski,C,37.9,STL,ATL,2000000.0,2015,746,1998.0,2016.0
4,A.J. Pierzynski,C,38.8,ATL,ATL,3000000.0,2016,746,1998.0,2016.0
...,...,...,...,...,...,...,...,...,...,...
556,Yasmani Grandal,C,30.2,LAD,MIL,18250000.0,2019,11368,2012.0,2021.0
557,Yasmani Grandal,C,31.0,MIL,CHW,18250000.0,2020,11368,2012.0,2021.0
558,Yonder Alonso,1B,30.7,SEA,CLE,8000000.0,2018,2530,2010.0,2019.0
559,Yuniesky Betancourt,1B,31.1,PHI,MIL,900000.0,2013,8585,2005.0,2013.0


In [74]:
# rename columns
salaryFinalDF.rename(columns={'year': 'year_fa',
                              'key_fangraphs': 'IDfg',
                              'mlb_played_first': 'first_played',
                              'mlb_played_last': 'last_played',
                              'to_tam': 'to_team'}, inplace=True)
salaryFinalDF

,name,position,age,from_team,to_team,avg_salary,year_fa,IDfg,first_played,last_played
0,A.J. Ellis,C,35.6,PHI,MIA,2500000.0,2017,5677,2008.0,2018.0
1,A.J. Pierzynski,C,37.5,BOS,STL,500000.0,2014,746,1998.0,2016.0
2,A.J. Pierzynski,C,35.9,CHW,TEX,7500000.0,2013,746,1998.0,2016.0
3,A.J. Pierzynski,C,37.9,STL,ATL,2000000.0,2015,746,1998.0,2016.0
4,A.J. Pierzynski,C,38.8,ATL,ATL,3000000.0,2016,746,1998.0,2016.0
...,...,...,...,...,...,...,...,...,...,...
556,Yasmani Grandal,C,30.2,LAD,MIL,18250000.0,2019,11368,2012.0,2021.0
557,Yasmani Grandal,C,31.0,MIL,CHW,18250000.0,2020,11368,2012.0,2021.0
558,Yonder Alonso,1B,30.7,SEA,CLE,8000000.0,2018,2530,2010.0,2019.0
559,Yuniesky Betancourt,1B,31.1,PHI,MIL,900000.0,2013,8585,2005.0,2013.0


## import mlb batting stats from 07-20 and clean data furthermore
1. Remove entries with BA less than 400:
2. Remove unwanted variables

In [75]:
# import mlb all batting stats from 2007-2020
batting_df = pd.read_csv('mlb-batting.csv')
batting_df.head()
print(batting_df.shape)
# drop rows with AB < 400
# batting_df = batting_df.drop(batting_df[batting_df['AB']<400].index)
# batting_df.shape

(13069, 320)


In [76]:
# drop all unwanted stats
pd.set_option('display.max_columns', None)
batting_df.head()

,Unnamed: 0,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,0,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,47,5,32,129,80,81,8,91,8,5,0,5,30,6,0.346,147,195,92,18,2582,1095,1487,14,0,0,0.132,0.148,0.89,0.438,0.640,1.078,0.294,0.368,0.0075,0.212,0.339,0.449,0.092,0.164,0.095,0.0,0.449,67.2,139,62.4,16.8,18.4,-5.3,101.1,10.4,$83.3,7.0,185,5.77,-8.16,13.93,62.93,6.11,0.92,3.62,4,6.44,-0.15,0.547,93.2,0.199,84.5,0.050,88.7,0.073,79.1,0.114,84.4,0.017,85.4,NaN,NaN,0.009,NaN,40.1,11.2,4.2,1.1,11.1,2.3,NaN,2.84,2.19,3.28,0.58,3.79,5.06,NaN,0.198,0.570,0.356,0.706,0.930,0.859,0.426,0.562,0.050,6.9,0.350,0.097,0.047,0.022,NaN,0.101,0.204,0.058,0.015,NaN,0.106,NaN,NaN,NaN,93.5,92.8,88.4,85.2,NaN,92.1,84.6,78.4,80.8,NaN,84.4,NaN,NaN,-2.4,-6.1,0.4,-4.1,NaN,-1.6,1.6,2.9,4.5,NaN,0.3,NaN,NaN,9.6,6.0,4.9,3.2,NaN,5.4,1.2,-5.3,-5.8,NaN,4.5,NaN,NaN,18.3,13.3,3.1,2.5,NaN,7.1,9.8,1.5,2.5,NaN,10.1,NaN,NaN,2.03,5.33,2.58,4.34,NaN,2.72,1.87,1.03,6.59,NaN,3.72,NaN,NaN,0.158,0.546,0.355,0.635,0.919,0.857,0.509,NaN,11.6,3.1,2.7,25 - 25,69.3,1.9,1.1,0.472,0.348,0.180,0.122,0.433,0.445,0.332,0.105,NaN,0.070,0.366,0.046,0.026,NaN,NaN,0.173,0.201,0.0,84.6,NaN,79.0,93.6,88.6,84.6,NaN,NaN,92.6,84.7,93.5,0.4,NaN,3.6,-1.9,0.5,-3.2,NaN,NaN,-4.2,2.1,-6.4,3.1,NaN,-7.2,8.2,4.2,1.6,NaN,NaN,4.4,-0.2,5.9,10.5,NaN,0.9,18.2,4.0,4.0,NaN,NaN,21.4,10.5,0.0,3.82,NaN,0.48,1.91,3.30,5.95,NaN,NaN,4.75,2.00,2.04,0.162,0.539,0.355,0.647,0.917,0.857,0.512,NaN,NaN,139.0,159,68,137.0,154.0,176.0,125.0,1.00,80.0,123.0,129.0,115.0,102.0,73.0,69.0,92.0,125.0,92.3,18.5,57.0,0.131,110.6,217.0,0.5,434,0.220,0.270,NaN,NaN,NaN
1,1,10155,2013,Mike Trout,LAA,21,157,589,716,190,115

In [77]:
var_list_basic = ['IDfg', 
            'Season', 
            'Name', 
            'Team', 
            'Age', 
            'G', 
            'PA', 
            'AB', 
            'R', 
            'H',
            '2B',
            '3B',
            'HR',
            'RBI',
            'SB',
            'CS',
            'BB',
            'SO',
            'GDP',
            'HBP',
            'SH',
            'SF',
            'IBB',
            'AVG',
            'OBP',
            'SLG',
            'BABIP',
            'ISO',
            'OPS',
            'wRC',
            'wRAA',
            'wOBA',
            'wRC+',
            'WPA',
            'WAR']
# not the full list
batting_basic_df = batting_df[var_list_basic]
batting_basic_df = batting_basic_df.sort_values(by=['IDfg', 'Season'], ascending=True)
batting_basic_df.reset_index(drop=True, inplace=True)
batting_basic_df

,IDfg,Season,Name,Team,Age,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,GDP,HBP,SH,SF,IBB,AVG,OBP,SLG,BABIP,ISO,OPS,wRC,wRAA,wOBA,wRC+,WPA,WAR
0,1,2007,Alfredo Amezaga,FLA,29,133,448,400,46,105,14,9,2,30,13,7,35,52,4,4,4,5,0,0.263,0.324,0.358,0.293,0.095,0.682,45,-9.9,0.305,79,-2.52,2.0
1,1,2008,Alfredo Amezaga,FLA,30,125,337,311,41,82,13,5,3,32,8,2,19,47,6,3,4,0,1,0.264,0.312,0.367,0.303,0.103,0.679,33,-7.5,0.301,77,-0.59,1.2
2,1,2009,Alfredo Amezaga,FLA,31,27,75,69,6,15,3,0,0,5,1,1,5,16,0,0,0,1,2,0.217,0.267,0.261,0.278,0.043,0.528,3,-6.2,0.228,28,-0.90,-0.2
3,1,2011,Alfredo Amezaga,- - -,33,40,87,77,6,14,1,0,0,4,0,0,7,14,1,0,2,1,0,0.182,0.247,0.195,0.219,0.013,0.442,2,-7.4,0.208,17,-0.85,-0.6
4,2,2007,Garret Anderson,LAA,35,108,450,417,67,124,31,1,16,80,1,0,27,54,8,0,0,6,9,0.297,0.336,0.492,0.306,0.194,0.827,62,6.3,0.348,110,-0.02,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13064,22581,2020,Alejandro Kirk,TOR,21,9,25,24,4,9,2,0,1,3,0,0,1,4,0,0,0,0,0,0.375,0.400,0.583,0.421,0.208,0.983,5,2.1,0.418,165,0.08,0.2
13065,23378,2020,Jose Barrero,CIN,22,24,68,67,4,13,0,0,0,2,1,1,1,26,1,0,0,0,0,0.194,0.206,0.194,0.317,0.000,0.400,0,-8.1,0.179,0,-0.98,-0.6
13066,24618,2020,Ryan Jeffers,MIN,23,26,62,55,5,15,0,0,3,7,0,0,5,19,0,2,0,0,0,0.273,0.355,0.436,0.364,0.164,0.791,9,1.4,0.346,120,-0.04,0.5
13067,27459,2020,Yoshi Tsutsugo,TBR,28,51,185,157,27,31,5,1,8,24,0,0,26,50,5,1,0,1,1,0.197,0.314,0.395,0.230,0.197,0.708,21,-1.7,0.309,99,0.01,0.3


# Merge salary and batting for EDA

In [78]:
# re_index batting_basic_df
batting_basic_df.reset_index(drop=True, inplace=True)
batting_basic_df = batting_basic_df.set_index(['IDfg'])
batting_basic_df.sort_index(level=['IDfg'], inplace=True)

In [79]:
# aggregate batting_basic based on FA year
agg_method = {'Age':'max',
             'G':'sum',
             'PA':'sum',
             'AB':'sum',
             'R':'sum',
             'H':'sum',
             '2B':'sum',
             '3B':'sum',
             'HR':'sum',
             'RBI':'sum',
             'SB':'sum',
             'CS':'sum',
             'BB':'sum',
             'SO':'sum',
             'GDP':'sum',
             'HBP':'sum',
             'SH':'sum',
             'SF':'sum',
             'IBB':'sum',
             'AVG':'mean',
             'OBP':'mean',
             'SLG':'mean',
             'BABIP':'mean',
             'ISO':'mean',
             'OPS':'mean',
             'wRC':'sum',
             'wRAA':'sum',
             'wRC+':'sum',
             'WPA':'sum',
             'WAR':'sum'}
batting_aggDF = pd.DataFrame()
for row in salaryFinalDF.itertuples():
    selected_years = [row.year_fa-1, row.year_fa-2, row.year_fa-3, row.year_fa-4, row.year_fa-5,]
    player = batting_basic_df.loc[row.IDfg]
    player = player[player['Season'].isin(selected_years)].groupby(by=['IDfg','Name']).agg(agg_method)
    player['Year_FA']=row.year_fa
    player['Salary']=row.avg_salary
    batting_aggDF = batting_aggDF.append(player)

In [80]:
# natural log salaries
batting_aggDF.insert(len(batting_aggDF.columns), 'Salary_log',
         np.log(batting_aggDF['Salary']))

In [81]:
batting_aggDF

Age    G    PA    AB    R    H   2B  3B   HR  RBI  \
IDfg  Name                                                                     
5677  A.J. Ellis            35  468  1713  1448  143  341   63   2   35  172   
746   A.J. Pierzynski       36  664  2587  2424  254  682  122   7   74  300   
      A.J. Pierzynski       35  664  2628  2455  272  695  129   7   70  290   
      A.J. Pierzynski       37  628  2414  2258  222  616  112   7   66  288   
      A.J. Pierzynski       38  613  2347  2191  217  610  107   8   66  281   
...                        ...  ...   ...   ...  ...  ...  ...  ..  ...  ...   
11368 Yasmani Grandal       29  638  2326  2000  254  471   95   4  104  294   
      Yasmani Grandal       30  663  2515  2136  286  512  102   5  117  322   
2530  Yonder Alonso         30  582  2118  1888  235  500  104   2   53  226   
8585  Yuniesky Betancourt   30  647  2498  2356  238  604  126  15   49  282   
2616  Zack Cozart           31  594  2390  2169  297  553  110  18   65  242   

                           SB  CS   BB   SO  GDP  HBP  SH  SF  IBB     AVG  \
IDfg  Name                                                                   
5677  A.J. Ellis            2   3  214  311   52   17  19  15   20  0.2306   
746   A.J. Pierzynski       5   6  101  278   76   29  12  21   21  0.2814   
      A.J. Pierzynski       5   5  109  273   76   28  15  21   24  0.2832   
      A.J. Pierzynski       4   6   91  280   71   33  10  22   17  0.2716   
      A.J. Pierzynski       1   4   95  278   73   34   4  23   17  0.2776   
...                        ..  ..  ...  ...  ...  ...  ..  ..  ...     ...   
11368 Yasmani Grandal       6   6  299  577   56    9   2  16    4  0.2350   
      Yasmani Grandal       8   7  350  601   65   12   2  15    5  0.2392   
2530  Yonder Alonso        19   7  204  323   54    9   0  17   16  0.2644   
8585  Yuniesky Betancourt  13  15   86  238   79    5  22  29    4  0.2526   
2616  Zack Cozart          17   4  164  372   49   16  20  21    9  0.2564   

                              OBP     SLG   BABIP     ISO     OPS  wRC  \
IDfg  Name                                                               
5677  A.J. Ellis           0.3340  0.3466  0.2680  0.1164  0.6804  183   
746   A.J. Pierzynski      0.3154  0.4288  0.2898  0.1476  0.7440  301   
      A.J. Pierzynski      0.3184  0.4270  0.2922  0.1440  0.7452  305   
      A.J. Pierzynski      0.3068  0.4112  0.2840  0.1398  0.7180  268   
      A.J. Pierzynski      0.3146  0.4196  0.2904  0.1422  0.7342  274   
...                           ...     ...     ...     ...     ...  ...   
11368 Yasmani Grandal      0.3352  0.4412  0.2742  0.2060  0.7764  309   
      Yasmani Grandal      0.3458  0.4546  0.2746  0.2154  0.8004  359   
2530  Yonder Alonso        0.3336  0.4028  0.2912  0.1384  0.7366  255   
8585  Yuniesky Betancourt  0.2778  0.3858  0.2594  0.1332  0.6632  224   
2616  Zack Cozart          0.3110  0.4226  0.2768  0.1660  0.7334  267   

                                   wRAA  wRC+   WPA   WAR  Year_FA  \
IDfg  Name                                                           
5677  A.J. Ellis          -8.400000e+00   464 -1.46   2.6     2017   
746   A.J. Pierzynski      6.000000e+00   473 -2.58   8.8     2014   
      A.J. Pierzynski     -1.665335e-16   468 -2.85   9.7     2013   
      A.J. Pierzynski     -2.400000e+00   452 -4.54   5.3     2015   
      A.J. Pierzynski      1.210000e+01   483 -2.46   5.1     2016   
...                                 ...   ...   ...   ...      ...   
11368 Yasmani Grandal      4.150000e+01   574 -1.00  22.3     2019   
      Yasmani Grandal      5.890000e+01   585  1.48  25.0     2020   
2530  Yonder Alonso        1.230000e+01   523  3.64   4.4     2018   
8585  Yuniesky Betancourt -6.740000e+01   374 -7.71  -2.8     2013   
2616  Zack Cozart         -4.900000e+00   469 -3.56  11.0     2018   

                               Salary  Salary_log  
IDfg  Name                                   